In [1]:
import pandas as pd
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [2]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [3]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [4]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [5]:
dependent=dataset[['charges']]

In [6]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent,dependent,test_size=0.20,random_state=0)

In [7]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [8]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [15]:
from sklearn.model_selection import GridSearchCV
grid_params= {"learning_rate":[1,10,50],
             "n_estimators":[10,50],
              "max_depth":[3,5,9]
             }
grid=GridSearchCV(LGBMRegressor(),grid_params,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


C:\Users\jayac\AppData\Local\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'learning_rate': [1, 10, 50], 'max_depth': [3, 5, 9],
                         'n_estimators': [10, 50]},
             verbose=3)

In [ ]:
# from sklearn.model_selection import GridSearchCV
# grid_params= {"learning_rate":[1,10,50],
#              "n_estimators":[10,50,100,1000],
#               "max_depth":[3,5,9],
#               "subsample":[0.3,0.5,0.9],
#               "colsample_bytree":[0.3,0.7,0.9],
#               "min_child_samples":[3,6,9]
#              }

In [16]:
# grid=GridSearchCV(LGBMRegressor(),grid_params,refit=True,verbose=3,n_jobs=-1)
# grid.fit(x_train,y_train)

In [17]:
result=grid.cv_results_
print("The resut is:{}".format(grid.best_params_))

The resut is:{'learning_rate': 1, 'max_depth': 3, 'n_estimators': 10}


In [18]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012192,0.002398,0.003398,0.000489,1,3,10,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",8.694790e-01,8.215729e-01,8.044741e-01,8.354622e-01,8.004203e-01,8.262817e-01,2.496736e-02,1
1,0.022386,0.001854,0.005197,0.001938,1,3,50,"{'learning_rate': 1, 'max_depth': 3, 'n_estima...",8.431889e-01,7.970620e-01,7.764285e-01,8.242526e-01,7.570799e-01,7.996024e-01,3.117390e-02,3
2,0.012991,0.001264,0.003999,0.001095,1,5,10,"{'learning_rate': 1, 'max_depth': 5, 'n_estima...",8.469200e-01,8.212143e-01,7.897689e-01,8.394639e-01,7.691590e-01,8.133052e-01,2.959606e-02,2
3,0.030379,0.002243,0.004198,0.000399,1,5,50,"{'learning_rate': 1, 'max_depth': 5, 'n_estima...",8.142741e-01,7.465407e-01,7.438286e-01,7.949325e-01,7.170415e-01,7.633235e-01,3.577380e-02,5
4,0.017989,0.002190,0.003198,0.000399,1,9,10,"{'learning_rate': 1, 'max_depth': 9, 'n_estima...",8.180474e-01,7.902534e-01,7.693942e-01,8.117915e-01,7.548609e-01,7.888695e-01,2.414355e-02,4
5,0.043572,0.002058,0.004398,0.000490,1,9,50,"{'learning_rate': 1, 'max_depth': 9, 'n_estima...",7.697506e-01,7.312210e-01,7.198433e-01,7.711488e-01,7.044795e-01,7.392887e-01,2.682492e-02,6
6,0.011392,0.001855,0.004398,0.002331,10,3,10,"{'learning_rate': 10, 'max_depth': 3, 'n_estim...",-1.132815e+19,-1.016885e+19,-1.084224e+19,-9.659667e+18,-9.833504e+18,-1.036648e+19,6.283088e+17,7
7,0.019188,0.001599,0.003199,0.000399,10,3,50,"{'learning_rate': 10, 'max_depth': 3, 'n_estim...",-4.729001e+68,-4.245045e+68,-4.526158e+68,-4.032483e+68,-4.105052e+68,-4.327548e+68,2.622918e+67,13
8,0.013791,0.000747,0.003598,0.000490,10,5,10,"{'learning_rate': 10, 'max_depth': 5, 'n_estim...",-1.121479e+19,-1.045620e+19,-1.097845e+19,-9.869578e+18,-9.824082e+18,-1.046862e+19,5.641149e+17,8
9,0.030979,0.004287,0.004798,0.001599,10,5,50,"{'learning_rate': 10, 'max_depth': 5, 'n_estim...",-4.681678e+68,-4.365001e+68,-4.583018e+68,-4.120112e+68,-4.101120e+68,-4.370186e+68,2.354935e+67,14


In [19]:
'age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'
age=int(input("Enter the Age: "))
bmi=int(input("Enter the BMI_range: "))
children=int(input("Enter the Children_count: "))
sex_male=int(input("Enter the sex, male 1/female 0: "))
smoker_yes=int(input("Enter the smoker, yes 1/no 0): "))

pred=grid.predict([[age,bmi,children,sex_male,smoker_yes]])
print("Prediction is: {}".format(pred))

Enter the Age: 23
Enter the BMI_range: 25
Enter the Children_count: 1
Enter the sex, male 1/female 0: 1
Enter the smoker, yes 1/no 0): 1
Prediction is: [53395.55732423]
